# IceCube Data Visualization Test

이 노트북은 GENESIS 프로젝트의 데이터를 로드하고 시각화하는 테스트를 위한 것입니다.

## 목차
1. [환경 설정 및 라이브러리 임포트](#1-환경-설정-및-라이브러리-임포트)
2. [데이터 로딩](#2-데이터-로딩)
3. [기본 데이터 탐색](#3-기본-데이터-탐색)
4. [3D 이벤트 시각화](#4-3d-이벤트-시각화)
5. [히스토그램 분석](#5-히스토그램-분석)
6. [NPZ 파일 시각화](#6-npz-파일-시각화)
7. [고급 시각화](#7-고급-시각화)


## 1. 환경 설정 및 라이브러리 임포트


In [1]:
# 필요한 라이브러리 임포트
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pandas as pd
from pathlib import Path
import torch

# 프로젝트 루트를 Python 경로에 추가
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# GENESIS 유틸리티 임포트
from utils.denormalization import denormalize_signal, denormalize_label, denormalize_full_event
from utils.fast_3d_plot import plot_event_3d, plot_event_comparison
from utils.h5_hist import plot_hist_pair
from utils.npz_show_event import show_event
from utils.visualization import create_3d_event_plot, EventVisualizer

# 설정
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
print("✅ 라이브러리 임포트 완료!")
print(f"📁 프로젝트 루트: {project_root}")
print(f"🐍 Python 경로: {sys.path[0]}")


ModuleNotFoundError: No module named 'utils.h5_reader'

## 2. 데이터 로딩


In [ ]:
# 데이터 경로 설정
data_path = project_root / "configs" / "default.yaml"
detector_csv = project_root / "configs" / "detector_geometry.csv"

# 설정 파일 로드
from config import load_config_from_file

try:
    config = load_config_from_file(str(data_path))
    print("✅ 설정 파일 로드 성공!")
    print(f"📊 데이터셋 경로: {config.data.h5_path}")
    print(f"🔄 시간 변환: {config.data.time_transform}")
    print(f"📏 시퀀스 길이: {config.model.seq_len}")
except Exception as e:
    print(f"❌ 설정 파일 로드 실패: {e}")
    # 기본값 설정
    config = None

# 검출기 지오메트리 로드
try:
    geometry_df = pd.read_csv(detector_csv)
    geometry = geometry_df[['x', 'y', 'z']].values
    print(f"✅ 검출기 지오메트리 로드 성공! Shape: {geometry.shape}")
    print(f"📍 X 범위: [{geometry[:, 0].min():.1f}, {geometry[:, 0].max():.1f}]")
    print(f"📍 Y 범위: [{geometry[:, 1].min():.1f}, {geometry[:, 1].max():.1f}]")
    print(f"📍 Z 범위: [{geometry[:, 2].min():.1f}, {geometry[:, 2].max():.1f}]")
except Exception as e:
    print(f"❌ 검출기 지오메트리 로드 실패: {e}")
    geometry = None


In [ ]:
# HDF5 데이터 로드 (실제 데이터가 있는 경우)
if config and os.path.exists(config.data.h5_path):
    try:
        with h5py.File(config.data.h5_path, 'r') as f:
            print("📁 HDF5 파일 구조:")
            for key in f.keys():
                print(f"  - {key}: {f[key].shape}")
            
            # 첫 번째 이벤트 로드
            if 'input' in f:
                sample_input = f['input'][0]  # (2, 5160)
                print(f"✅ 샘플 입력 데이터 로드: {sample_input.shape}")
                print(f"  - Charge 범위: [{sample_input[0].min():.3f}, {sample_input[0].max():.3f}]")
                print(f"  - Time 범위: [{sample_input[1].min():.3f}, {sample_input[1].max():.3f}]")
            
            if 'label' in f:
                sample_label = f['label'][0]  # (6,)
                print(f"✅ 샘플 라벨 데이터 로드: {sample_label.shape}")
                print(f"  - Energy: {sample_label[0]:.3f}")
                print(f"  - Zenith: {sample_label[1]:.3f}")
                print(f"  - Azimuth: {sample_label[2]:.3f}")
                print(f"  - Position: ({sample_label[3]:.1f}, {sample_label[4]:.1f}, {sample_label[5]:.1f})")
                
    except Exception as e:
        print(f"❌ HDF5 데이터 로드 실패: {e}")
        sample_input = None
        sample_label = None
else:
    print("⚠️ HDF5 데이터 파일을 찾을 수 없습니다. 더미 데이터를 생성합니다.")
    # 더미 데이터 생성
    sample_input = np.random.exponential(0.1, (2, 5160))
    sample_input[1] = np.random.normal(500, 100, 5160)  # Time
    sample_label = np.array([1.0, 0.5, 0.0, 0.0, 0.0, 0.0])  # [energy, zenith, azimuth, x, y, z]
    print("✅ 더미 데이터 생성 완료!")


## 3. 기본 데이터 탐색


In [ ]:
# 기본 통계 분석
if sample_input is not None:
    charge_data = sample_input[0]
    time_data = sample_input[1]
    
    print("📊 데이터 통계 분석")
    print("=" * 50)
    
    # Charge 통계
    print("🔋 Charge (NPE) 통계:")
    print(f"  - 총 PMT 수: {len(charge_data)}")
    print(f"  - 0이 아닌 PMT 수: {np.sum(charge_data > 0)} ({100*np.sum(charge_data > 0)/len(charge_data):.1f}%)")
    print(f"  - 평균: {np.mean(charge_data):.4f}")
    print(f"  - 중앙값: {np.median(charge_data):.4f}")
    print(f"  - 표준편차: {np.std(charge_data):.4f}")
    print(f"  - 범위: [{np.min(charge_data):.4f}, {np.max(charge_data):.4f}]")
    
    print("\n⏰ Time 통계:")
    finite_time = time_data[np.isfinite(time_data)]
    print(f"  - 유한한 값의 수: {len(finite_time)} ({100*len(finite_time)/len(time_data):.1f}%)")
    if len(finite_time) > 0:
        print(f"  - 평균: {np.mean(finite_time):.2f} ns")
        print(f"  - 중앙값: {np.median(finite_time):.2f} ns")
        print(f"  - 표준편차: {np.std(finite_time):.2f} ns")
        print(f"  - 범위: [{np.min(finite_time):.2f}, {np.max(finite_time):.2f}] ns")
    
    # 간단한 히스토그램
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Charge 히스토그램
    charge_nonzero = charge_data[charge_data > 0]
    if len(charge_nonzero) > 0:
        ax1.hist(charge_nonzero, bins=50, alpha=0.7, color='blue', edgecolor='black')
        ax1.set_xlabel('Charge (NPE)')
        ax1.set_ylabel('Count')
        ax1.set_title('Charge Distribution (Non-zero)')
        ax1.set_yscale('log')
        ax1.grid(True, alpha=0.3)
    
    # Time 히스토그램
    if len(finite_time) > 0:
        ax2.hist(finite_time, bins=50, alpha=0.7, color='green', edgecolor='black')
        ax2.set_xlabel('Time (ns)')
        ax2.set_ylabel('Count')
        ax2.set_title('Time Distribution')
        ax2.set_yscale('log')
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ 샘플 데이터가 없습니다.")


## 4. 3D 이벤트 시각화


In [ ]:
# 3D 이벤트 시각화 (fast_3d_plot 사용)
if sample_input is not None and geometry is not None and sample_label is not None:
    print("🎨 3D 이벤트 시각화 생성 중...")
    
    # fast_3d_plot을 사용한 시각화
    fig, axes = plot_event_3d(
        charge_data=sample_input[0],
        time_data=sample_input[1], 
        geometry=geometry,
        labels=sample_label,
        output_path=None,  # 노트북에서 표시
        plot_type="both",  # NPE와 Time 모두 표시
        figure_size=(20, 10),
        show_detector_hull=True,
        show_background=True,
        sphere_size=3.0,
        alpha=0.8
    )
    
    plt.show()
    print("✅ 3D 시각화 완료!")
else:
    print("❌ 3D 시각화를 위한 데이터가 부족합니다.")
    print(f"  - sample_input: {sample_input is not None}")
    print(f"  - geometry: {geometry is not None}")
    print(f"  - sample_label: {sample_label is not None}")


## 5. 히스토그램 분석


In [ ]:
# h5_hist를 사용한 아름다운 히스토그램 생성
if config and os.path.exists(config.data.h5_path):
    print("📊 HDF5 데이터에서 히스토그램 생성 중...")
    
    try:
        # h5_hist를 사용한 히스토그램 생성
        plot_hist_pair(
            h5_path=config.data.h5_path,
            bins=200,
            chunk=1024,
            out_prefix="testing/hist_analysis",
            logy=True,
            log_time_transform="ln",
            exclude_zero=True,
            style="modern",
            figsize=(16, 8)
        )
        print("✅ 히스토그램 생성 완료! testing/ 폴더를 확인하세요.")
        
    except Exception as e:
        print(f"❌ 히스토그램 생성 실패: {e}")
        
else:
    print("⚠️ HDF5 파일이 없어서 더미 데이터로 히스토그램을 생성합니다.")
    
    # 더미 데이터로 히스토그램 생성
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Charge 히스토그램 (더미 데이터)
    charge_dummy = np.random.exponential(0.1, 10000)
    charge_dummy = charge_dummy[charge_dummy > 0]  # 0 제외
    
    ax1.hist(charge_dummy, bins=50, alpha=0.7, color='#2E86AB', edgecolor='#1A5F7A')
    ax1.set_xlabel('Charge (NPE)')
    ax1.set_ylabel('Count (log scale)')
    ax1.set_title('Charge Distribution (Dummy Data)')
    ax1.set_yscale('log')
    ax1.grid(True, alpha=0.3)
    
    # Time 히스토그램 (더미 데이터)
    time_dummy = np.random.normal(500, 100, 10000)
    time_dummy = time_dummy[time_dummy > 0]  # 양수만
    
    ax2.hist(time_dummy, bins=50, alpha=0.7, color='#A23B72', edgecolor='#7D2A5A')
    ax2.set_xlabel('Time (ns)')
    ax2.set_ylabel('Count (log scale)')
    ax2.set_title('Time Distribution (Dummy Data)')
    ax2.set_yscale('log')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    print("✅ 더미 데이터 히스토그램 생성 완료!")


## 6. NPZ 파일 시각화


In [ ]:
# NPZ 파일 생성 및 시각화
if sample_input is not None and sample_label is not None:
    print("💾 NPZ 파일 생성 중...")
    
    # NPZ 파일 생성
    npz_path = "testing/sample_event.npz"
    np.savez(npz_path, 
             input=sample_input,
             label=sample_label,
             info=sample_label)  # info는 label과 동일하게 설정
    
    print(f"✅ NPZ 파일 생성 완료: {npz_path}")
    
    # NPZ 파일 시각화
    if geometry is not None:
        print("🎨 NPZ 파일 시각화 중...")
        
        try:
            fig, ax = show_event(
                npz_path=npz_path,
                detector_csv=str(detector_csv),
                out_path="testing/npz_visualization.png",
                sphere_res=(30, 15),
                base_radius=6.0,
                radius_scale=0.3,
                figure_size=(18, 12),
                separate_plots=False
            )
            plt.show()
            print("✅ NPZ 시각화 완료!")
            
        except Exception as e:
            print(f"❌ NPZ 시각화 실패: {e}")
    else:
        print("❌ 검출기 지오메트리가 없어서 NPZ 시각화를 할 수 없습니다.")
        
else:
    print("❌ NPZ 파일 생성을 위한 데이터가 부족합니다.")


## 7. 고급 시각화


In [ ]:
# EventVisualizer를 사용한 고급 시각화
if sample_input is not None and sample_label is not None and geometry is not None:
    print("🎨 EventVisualizer를 사용한 고급 시각화...")
    
    try:
        # EventVisualizer 초기화
        visualizer = EventVisualizer(
            detector_geometry_path=str(detector_csv),
            sphere_resolution=(40, 20),
            base_radius=5.0,
            radius_scale=0.2,
            figure_size=(20, 12),
            time_transform="ln",
            exclude_zero_time=True
        )
        
        # NPZ 파일이 있다면 시각화
        npz_path = "testing/sample_event.npz"
        if os.path.exists(npz_path):
            print("📊 NPZ 파일을 사용한 시각화...")
            fig, ax = visualizer.visualize_event(
                npz_path=npz_path,
                output_path="testing/advanced_visualization.png"
            )
            plt.show()
            print("✅ 고급 시각화 완료!")
        else:
            print("⚠️ NPZ 파일이 없어서 고급 시각화를 건너뜁니다.")
            
    except Exception as e:
        print(f"❌ 고급 시각화 실패: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ 고급 시각화를 위한 데이터가 부족합니다.")


## 8. 정규화/비정규화 테스트


In [ ]:
# 정규화/비정규화 테스트
if config and sample_input is not None:
    print("🔄 정규화/비정규화 테스트...")
    
    try:
        # 정규화된 데이터를 원래 스케일로 변환
        x_sig_original = denormalize_signal(
            torch.tensor(sample_input),
            affine_offsets=tuple(config.data.affine_offsets),
            affine_scales=tuple(config.data.affine_scales),
            time_transform=config.data.time_transform,
            channels="signal"
        )
        
        print("✅ 신호 비정규화 완료!")
        print(f"  - 원본 범위: Charge [{sample_input[0].min():.3f}, {sample_input[0].max():.3f}]")
        print(f"  - 비정규화 범위: Charge [{x_sig_original[0].min():.3f}, {x_sig_original[0].max():.3f}]")
        
        # 라벨 비정규화
        if sample_label is not None:
            label_original = denormalize_label(
                torch.tensor(sample_label),
                label_offsets=tuple(config.data.label_offsets),
                label_scales=tuple(config.data.label_scales)
            )
            
            print("✅ 라벨 비정규화 완료!")
            print(f"  - 원본 Energy: {sample_label[0]:.3f}")
            print(f"  - 비정규화 Energy: {label_original[0]:.3f}")
            
            # 비정규화된 데이터로 시각화
            if geometry is not None:
                print("🎨 비정규화된 데이터로 시각화...")
                fig, axes = plot_event_3d(
                    charge_data=x_sig_original[0].numpy(),
                    time_data=x_sig_original[1].numpy(),
                    geometry=geometry,
                    labels=label_original.numpy(),
                    output_path="testing/denormalized_visualization.png",
                    plot_type="both",
                    figure_size=(20, 10)
                )
                plt.show()
                print("✅ 비정규화 시각화 완료!")
        
    except Exception as e:
        print(f"❌ 정규화/비정규화 테스트 실패: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ 정규화/비정규화 테스트를 위한 설정이나 데이터가 부족합니다.")


## 9. 요약 및 다음 단계

이 노트북을 통해 다음을 테스트했습니다:

### ✅ 완료된 테스트
1. **환경 설정**: 라이브러리 임포트 및 경로 설정
2. **데이터 로딩**: 설정 파일, 검출기 지오메트리, HDF5 데이터
3. **기본 탐색**: 데이터 통계 분석 및 히스토그램
4. **3D 시각화**: fast_3d_plot을 사용한 이벤트 시각화
5. **히스토그램 분석**: h5_hist를 사용한 아름다운 히스토그램
6. **NPZ 시각화**: NPZ 파일 생성 및 시각화
7. **고급 시각화**: EventVisualizer를 사용한 고급 기능
8. **정규화/비정규화**: 데이터 변환 테스트

### 📁 생성된 파일들
- `testing/sample_event.npz`: 샘플 이벤트 NPZ 파일
- `testing/npz_visualization.png`: NPZ 시각화 이미지
- `testing/advanced_visualization.png`: 고급 시각화 이미지
- `testing/denormalized_visualization.png`: 비정규화 시각화 이미지
- `testing/hist_analysis_*.png`: 히스토그램 분석 이미지들

### 🚀 다음 단계 제안
1. **실제 데이터로 테스트**: HDF5 파일이 있다면 실제 데이터로 테스트
2. **다양한 이벤트 시각화**: 여러 이벤트를 비교 시각화
3. **커스텀 시각화**: 특정 요구사항에 맞는 시각화 개발
4. **성능 최적화**: 대용량 데이터 처리 최적화
5. **인터랙티브 시각화**: Plotly 등을 사용한 인터랙티브 시각화
